In [1]:
# create species annotation class
import os
import pickle
import sys
PROJ_DIR = "/Users/woosubs/Desktop/AutomateAnnotation/AnnotationRecommender/"
MOD_DIR = os.path.join(PROJ_DIR, "annotation_recommender")
sys.path.append(MOD_DIR)

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")


from annotation_recommender import species_annotation as sa
from annotation_recommender import constants as cn

with open(os.path.join(CHEBI_DIR, 'chebi_shortened_formula_30apr2022.pickle'), 'rb') as f:
  ref_shortened_chebi_to_formula = pickle.load(f)

with open(os.path.join(CHEBI_DIR, 'chebi_synonyms.pickle'), 'rb') as f:
  chebi_synonyms = pickle.load(f)
chebi_low_synonyms = dict()
for one_k in chebi_synonyms.keys():
  chebi_low_synonyms[one_k] = list(set([val.lower() for val in chebi_synonyms[one_k]]))

with open(os.path.join(RHEA_DIR, 'kegg2rhea_master.pickle'), 'rb') as handle:
  ref_kegg2rhea_master = pickle.load(handle)
with open(os.path.join(RHEA_DIR, 'kegg2rhea_bi.pickle'), 'rb') as handle:
  ref_kegg2rhea_bi = pickle.load(handle)

# load reference matrix
with open(os.path.join(ALGO_DIR, 'binary_ref_df.pickle'), 'rb') as handle:
    ref_mat = pickle.load(handle)
# check its shape
print(ref_mat.shape)

(13651, 3790)


In [2]:
specanot = sa.SpeciesAnnotation(libsbml_fpath=ecoli_fpath)

In [3]:
print(specanot.spec_dict)

{'M_glc__D_e': ['C6O6'], 'M_gln__L_c': ['C5N2O3'], 'M_gln__L_e': ['C5N2O3'], 'M_glu__L_c': ['C5NO4'], 'M_glu__L_e': ['C5NO4'], 'M_glx_c': ['C2O3'], 'M_h2o_c': ['[18O]', 'O'], 'M_h2o_e': ['[18O]', 'O'], 'M_h_c': ['H'], 'M_h_e': ['H'], 'M_icit_c': ['C6O7'], 'M_lac__D_c': ['C3O3'], 'M_lac__D_e': ['C3O3'], 'M_mal__L_c': ['C4O5'], 'M_mal__L_e': ['C4O5'], 'M_nad_c': ['C21N7O14P2'], 'M_nadh_c': ['C21N7O14P2'], 'M_nadp_c': ['C21N7O17P3'], 'M_nadph_c': ['C21N7O17P3'], 'M_nh4_c': ['N', '[13N]'], 'M_13dpg_c': ['C3O10P2'], 'M_nh4_e': ['N', '[13N]'], 'M_o2_c': ['O2'], 'M_2pg_c': ['C3O7P'], 'M_o2_e': ['O2'], 'M_3pg_c': ['C3O7P'], 'M_oaa_c': ['C4O5'], 'M_pep_c': ['C3O6P'], 'M_6pgc_c': ['C6O10P'], 'M_pi_c': ['O4P'], 'M_6pgl_c': ['C6O9P'], 'M_pi_e': ['O4P'], 'M_ac_c': ['C2O2'], 'M_pyr_c': ['C3O3'], 'M_pyr_e': ['C3O3'], 'M_q8_c': ['C49O4'], 'M_q8h2_c': ['C49O4'], 'M_r5p_c': ['C5O8P'], 'M_ru5p__D_c': ['C5O8P'], 'M_ac_e': ['C2O2'], 'M_acald_c': ['C2O'], 'M_s7p_c': ['C7O10P'], 'M_acald_e': ['C2O'], 'M_acco

In [4]:
candidates_name = specanot.predictSpeciesCandidatesByName(['M_glc__D_e'])

In [5]:
candidates_name

{'M_glc__D_e': {'match_score': 1.0,
  'chebi': ['CHEBI:17634', 'CHEBI:4167', 'CHEBI:42758'],
  'formula2chebi': {'C6O6': ['CHEBI:17634', 'CHEBI:4167', 'CHEBI:42758']},
  'formula': ['C6O6']}}